In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(f"device is {device}")

device is cuda:0


# Without AutoGrad

In [2]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(1000):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 310.68438720703125
199 0.6918531656265259
299 0.002619071863591671
399 9.210624557454139e-05
499 2.135093200195115e-05
599 9.860440513875801e-06
699 6.22737206867896e-06
799 4.342710781202186e-06
899 3.272908315921086e-06
999 2.6395623535790946e-06


# With AutoGrad

In [3]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(1000):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss)

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

99 tensor(671.7351, device='cuda:0', grad_fn=<SumBackward0>)
199 tensor(5.1488, device='cuda:0', grad_fn=<SumBackward0>)
299 tensor(0.1060, device='cuda:0', grad_fn=<SumBackward0>)
399 tensor(0.0036, device='cuda:0', grad_fn=<SumBackward0>)
499 tensor(0.0003, device='cuda:0', grad_fn=<SumBackward0>)
599 tensor(6.5771e-05, device='cuda:0', grad_fn=<SumBackward0>)
699 tensor(2.7854e-05, device='cuda:0', grad_fn=<SumBackward0>)
799 tensor(1.6614e-05, device='cuda:0', grad_fn=<SumBackward0>)
899 tensor(1.1648e-05, device='cuda:0', grad_fn=<SumBackward0>)
999 tensor(8.8776e-06, device='cuda:0', grad_fn=<SumBackward0>)


# Pytorch package: nn

In [4]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device=device)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 tensor(1.7060, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(0.0364, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0026, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(0.0002, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(2.4472e-05, device='cuda:0', grad_fn=<MseLossBackward>)


# Pytorch package: optim

In [5]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device=device)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 tensor(38.8244, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(0.3023, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0021, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(1.4786e-05, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(8.7089e-08, device='cuda:0', grad_fn=<MseLossBackward>)


# Pytorch package: nn.Module

In [6]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.relu    = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred

model = TwoLayerNet(D_in, H, D_out)
model.to(device=device)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 tensor(1.7543, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(0.0313, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(6.1273e-05, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(3.3246e-06, device='cuda:0', grad_fn=<MseLossBackward>)
